# -----------------American Options (Quyền chọn kiểu MỸ)-----------------------
1,Quyền chọn kiểu Mỹ vs. Quyền chọn kiểu Châu Âu:

-Quyền chọn kiểu Châu Âu chỉ có thể được thực hiện vào ngày đáo hạn.
-Quyền chọn kiểu Mỹ có thể được thực hiện bất cứ lúc nào trước và bao gồm ngày đáo hạn.
Điều này làm cho việc định giá phức tạp hơn vì phải xem xét nhiều điểm dừng (exercise points) có thể xảy ra.

2,Vấn đề dừng tối ưu (Optimal Stopping Problem):
Để định giá quyền chọn kiểu Mỹ, cần giải quyết một bài toán dừng tối ưu. Điều này có nghĩa là phải xác định
thời điểm tốt nhất để thực hiện quyền chọn nhằm tối đa hóa giá trị của nó.
Phương trình 12-12 trong tài liệu được đề cập mô tả cách định giá quyền chọn kiểu Mỹ dưới dạng một bài toán dừng tối ưu.

3,Lưới thời gian rời rạc (Discrete Time Grid):
Việc định giá dựa trên một lưới thời gian rời rạc (discrete time grid) nghĩa là thời gian 
được chia thành các khoảng nhỏ, và việc định giá được thực hiện tại mỗi điểm thời gian này.
Đây là cơ sở để sử dụng mô phỏng số (numerical simulation) để tính toán giá trị quyền chọn.

4,Quyền chọn kiểu Bermudan:
Quyền chọn kiểu Bermudan là một loại quyền chọn có thể được thực hiện tại một số thời điểm xác định trước trong suốt thời gian sống của quyền chọn, 
chứ không phải bất kỳ lúc nào như quyền chọn kiểu Mỹ.
Khi thời gian giữa các điểm dừng này trở nên rất nhỏ (time interval converging to zero length), giá trị của quyền chọn kiểu Bermudan sẽ tiến gần đến 
giá trị của quyền chọn kiểu Mỹ.

Tóm tắt:
Định giá quyền chọn kiểu Mỹ phức tạp hơn quyền chọn kiểu Châu Âu vì cần giải quyết bài toán dừng tối ưu.
Phương pháp định giá này dựa trên lưới thời gian rời rạc và mô phỏng số.
Khi khoảng thời gian giữa các điểm dừng trở nên rất nhỏ, giá trị của quyền chọn kiểu Bermudan sẽ tiệm cận giá trị của quyền chọn kiểu Mỹ.
Như vậy, đoạn văn nhấn mạnh sự khác biệt trong việc định giá quyền chọn kiểu Mỹ và cách tiếp cận để giải quyết vấn đề này 
bằng cách sử dụng quyền chọn kiểu Bermudan như một bước đệm trong việc sử dụng mô phỏng số.

Ví dụ minh họa
Giả sử chúng ta có một quyền chọn mua kiểu Mỹ. Chúng ta cần xác định liệu có nên thực hiện quyền chọn ngay bây giờ hay chờ thêm một khoảng thời gian nữa. Điều này phụ thuộc vào việc so sánh giá trị tức thời của quyền chọn (giá tài sản cơ sở trừ đi giá thực hiện) với giá trị kỳ vọng khi giữ quyền chọn đến thời điểm tiếp theo.

1,Phương pháp cây nhị phân:
    Chia thời gian từ hiện tại đến ngày đáo hạn thành các khoảng nhỏ.
    Tại mỗi nút trên cây, tính giá trị quyền chọn tại mỗi bước dựa trên các khả năng lên hoặc xuống của giá tài sản cơ sở.
    Quyết định tối ưu tại mỗi nút là chọn giá trị lớn nhất giữa việc thực hiện quyền chọn ngay và giá trị kỳ vọng của quyền chọn ở các bước sau.
2,Phương pháp Monte Carlo:
    Mô phỏng nhiều con đường giá của tài sản cơ sở từ hiện tại đến ngày đáo hạn.
    Tại mỗi bước thời gian, so sánh giá trị tức thời của việc thực hiện quyền chọn với giá trị kỳ vọng khi giữ quyền chọn.
    Tính toán giá trị trung bình của quyền chọn từ các con đường giá để xác định giá trị của quyền chọn kiểu Mỹ.

In [16]:
import math
import numpy as np
import numpy.random as npr
S0 = 100.
r = 0.05
sigma = 0.25
T = 1.0
I = 50000
M = 50
def gen_sn(M, I, anti_paths=True, mo_match=True):
    ''' Function to generate random numbers for simulation.
    Parameters
    ==========
    M: int
    number of time intervals for discretization
    I: int
    number of paths to be simulated
    anti_paths: boolean
    use of antithetic variates
    mo_math: boolean
    use of moment matching
    '''
    if anti_paths is True:
        sn = npr.standard_normal((M + 1, int(I / 2)))
        sn = np.concatenate((sn, -sn), axis=1)
    else:
        sn = npr.standard_normal((M + 1, I))
    if mo_match is True:
        sn = (sn - sn.mean()) / sn.std()
    return sn
def gbm_mcs_amer(K, option='call'):
    ''' Valuation of American option in Black-Scholes-Merton
    by Monte Carlo simulation by LSM algorithm
    Parameters
    ==========
    K: float
    (positive) strike price of the option
    option: string
    type of the option to be valued ('call', 'put')
    Returns
    =======
    C0: float
    estimated present value of American call option
    '''
    dt = T / M
    df = math.exp(-r * dt)
    # simulation of index levels
    S = np.zeros((M + 1, I))
    S[0] = S0
    sn = gen_sn(M, I)
    for t in range(1, M + 1):
        S[t] = S[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt
        + sigma * math.sqrt(dt) * sn[t])
    # case based calculation of payoff
    print('S-K shape:',(S-K).shape)
    print('------>')
    if option == 'call':
        h = np.maximum(S - K, 0)
    else:
        h = np.maximum(K - S, 0)
    # Đến chỗ này, h là ma trận giá quyền chọn có shape = (51,50000), tiếp theo là thuật toán LMS
    # LSM algorithm
    V = np.copy(h)
    # Duyệt ngược từ 49 về 0
    for t in range(M - 1, 0, -1):
        # reg là bộ chỉ số cho đa thức bậc 7 sao cho hồi quy đa thức, tức là V[t+1] * df sẽ bằng một dạng đa thức bậc 7 nào đó của S[t], vè ý nghĩa thì V[t+1]*df là giá trị quyền chọn tai thời điểm t+1, đã được chiết khấu về 
        # thời điểm t
        reg = np.polyfit(S[t], V[t + 1] * df, 7)
        # Dựa vào bộ chỉ số reg, ta tính giá trị của đa thức y có chỉ số là reg tại x = S[t], nghĩa tính ngược trở lại  :v
        C = np.polyval(reg, S[t])
        # Cập nhật lại V[t] (giá trị quyền chọn), so sánh giá trị C(gtri mới, từng đoạn sau khi đã tính) với h[t](giá trị cũ) để quyết định xem liệu quyền được chọn hay giữ lại
        # Nếu giá trị C > h[t], thì V[t] = V[t+1]*df
        # Nếu C < h[t] thì V[t] = h[t]
        V[t] = np.where(C > h[t], V[t + 1] * df, h[t])
    # MCS estimator
    C0 = df * np.mean(V[1])
    return C0
a = gbm_mcs_amer(110.,option = 'call')
print(a)

S-K shape: (51, 50000)
------>
7.746964676110448
